# Problem 1:

LeetCode 966: https://leetcode.com/problems/vowel-spellchecker/

This is a simplified version for spell correction, since it only checks two common types of spell errors. Spell correction is widely used in many cases, like search engine and word processing.

## Description

Given a wordlist, we want to implement a spellchecker that converts a query word into a correct word.

For a given query word, the spell checker handles two categories of spelling mistakes:

- Capitalization: If the query matches a word in the wordlist (case-insensitive), then the query word is returned with the same case as the case in the wordlist.

- Vowel Errors: If after replacing the vowels ('a', 'e', 'i', 'o', 'u') of the query word with any vowel individually, it matches a word in the wordlist (case-insensitive), then the query word is returned with the same case as the match in the wordlist.

## My solution

In [9]:
import collections
import math

def find_word(word_dict: collections.defaultdict(list), word: str):
    """
    find if a word match any word in word dictionary

    Return:
        the exact match if exists
        first match otherwise
    """       
    final_word = ""

    word_lowcase = word.lower()
    if word_lowcase in word_dict:
        # grab all word candiates
        word_candidate_tuples = word_dict[word_lowcase]
        word_candidates = [word_tuple[0] for word_tuple in word_candidate_tuples]

        if word in word_candidates:
            # find the exact match
            final_word = word
        else:
            # find the first match
            final_word = word_candidates[0]

    return final_word
    
def generate_new_words(word: str) -> set:
    """
    generate all new words by replacing vowels in a word
    """

    vowel_list = ['a', 'e', 'i', 'o', 'u']

    word_lowcase = word.lower()

    # creat a set holding new word set 
    new_word_set = set()
    new_word_set.add(word_lowcase)

    for i in range(len(word_lowcase)):
        curr_char = word_lowcase[i]

        if curr_char not in vowel_list:
            continue

        for vowel in vowel_list:

            # generate new words by replacing the current letter with 
            # every vowel for every word in the new word set
            curr_word_set = new_word_set.copy()                
            for w in curr_word_set:
                new_word = w[:i] + vowel+ w[i+1:]
                new_word_set.add(new_word)

    return new_word_set

def find_first_match(word_set: set, word_dict):
    """
    given a set of words, find the matching word with smallest index
    """

    final_word = ''

    matching_word_lst = []       
    for w in word_set:
        if w in word_dict:
            matching_word_lst += word_dict[w]
    # print('matching_word_lst:', matching_word_lst) 

    idx_min = math.inf
    for word_tuple in matching_word_lst:
        w = word_tuple[0]
        idx = word_tuple[1]

        if idx < idx_min:
            idx_min = idx
            final_word = w

    return final_word


def spellchecker(wordlist: list, queries: list) -> list:  
    """
    Given a wordlist, converts a query word into a correct word.
    """

    # create a dictionary:
    # key: lower case of words in wordlist
    # value: list of tuples: (original words whose lower case matches key, index)
    word_dict = collections.defaultdict(list)

    for i in range(len(wordlist)):
        word = wordlist[i]
        word_dict[word.lower()].append((word, i))      
    # print(word_dict)

    res = []
    for word in queries:     
        # print(word, "***")
        final_word = ""
        
        # check if word without subtition appears in worddict:
        final_word = find_word(word_dict, word)

        if final_word:
            res.append(final_word)
            continue

        # check if substitue a vowl would find a match
        new_words = generate_new_words(word)
        final_word = find_first_match(new_words, word_dict)

        res.append(final_word)

    return res


In [10]:
wordlist = ["KiTe","kite","hare","Hare"]
queries = ["kite","Kite","KiTe","Hare","HARE","Hear","hear","keti","keet","keto"]
expect_output = ["kite","KiTe","KiTe","Hare","hare","","","KiTe","","KiTe"]

spellchecker(wordlist, queries)

['kite', 'KiTe', 'KiTe', 'Hare', 'hare', '', '', 'KiTe', '', 'KiTe']

In [11]:
wordlist = ["ae","aa"]
queries = ["UU"]
expect_output = ["ae"]

spellchecker(wordlist, queries)

['ae']

## Leetcode solution

We analyze the 3 cases that the algorithm needs to consider: when the query is an exact match, when the query is a match up to capitalization, and when the query is a match up to vowel errors.

In all 3 cases, we can use a hash table to query the answer.

- For the first case (exact match), we hold a set of words to efficiently test whether our query is in the set.
- For the second case (capitalization), we hold a hash table that converts the word from its lowercase version to the original word (with correct capitalization).
- For the third case (vowel replacement), we hold a hash table that converts the word from its lowercase version with the vowels masked out, to the original word.

In [ ]:
def spellchecker(self, wordlist, queries):
    def devowel(word):
        return "".join('*' if c in 'aeiou' else c
                       for c in word)

    words_perfect = set(wordlist)
    words_cap = {}
    words_vow = {}

    # create three dictionaries
    for word in wordlist:
        wordlow = word.lower()
        
        # only hash the first matching instance in the wordlist
        # so we will return the first word that satisfies the condition 
        words_cap.setdefault(wordlow, word) 
        words_vow.setdefault(devowel(wordlow), word)  

    def solve(query):
        if query in words_perfect:
            return query

        queryL = query.lower()
        if queryL in words_cap:
            return words_cap[queryL]

        queryLV = devowel(queryL)
        if queryLV in words_vow:
            return words_vow[queryLV]
        return ""

    return map(solve, queries)


- map() function returns a map object(which is an iterator) of the results after applying the given function to each item of a given iterable (list, tuple etc.)
    
- In Dictionary, setdefault() method returns the value of a key (if the key is in dictionary). If not, it inserts key with a value to the dictionary.

# Problem 2

LeetCode 72: Edit Distance (https://leetcode.com/problems/edit-distance/)

use cases:
- find the correction word with minimum changes to the misspelled word
- DNA sequence matching

## Description

Given two words word1 and word2, find the minimum number of operations required to convert word1 to word2.

You have the following 3 operations permitted on a word:

- Insert a character
- Delete a character
- Replace a character

## solutions

### naive solution: recursion

In [67]:
def edist(s1: str, s2: str) -> int:
    
    # edit distance between an empty string and other string
    if len(s1) == 0:
        return len(s2)   
    if len(s2) == 0:
        return len(s1)
    
    delta = 0 if s1[-1] == s2[-1] else 1
    
    return min(edist(s1[:-1], s2[:-1]) + delta,
               edist(s1, s2[:-1]) + 1,
               edist(s1[:-1], s2) + 1
               )

In [73]:
s1 = 'birthday cake'
s2 = 'birthday'

In [76]:
%time print(edist(s1, s2))

5
CPU times: user 6.55 s, sys: 18.2 ms, total: 6.57 s
Wall time: 6.59 s


### dynamic programming

avoid redunrant computing, create a matrix where string 1 labels the rows and string 2 labels the columns, and each element corresponds to the edit distance of a particular prefix of string 1 and a particular prefix of string 2

```

      '' s2[0] s2[1] ... s2[-1]
''    
s1[0]
s1[1]
.
.
.
s1[-1]
```

In [78]:
def edist(s1: str, s2: str) -> int:
    
    nrow = len(s1) + 1
    ncol = len(s2) + 1
    
    # create empty matrix
    matrix = [[0]*ncol for i in range(nrow)]
    
    # fill in the first row and column
    for i in range(nrow):
        matrix[i][0] = i
    for i in range(ncol):
        matrix[0][i] = i 
    
    for i in range(1, nrow):
        for j in range(1, ncol):
            
            delta = 0 if s1[i-1] == s2[j-1] else 1
            matrix[i][j] = min(matrix[i-1][j-1] + delta,
                               matrix[i][j-1] + 1,
                               matrix[i-1][j] + 1
                               )
    return matrix[-1][-1]

In [79]:
s1 = 'birthday cake'
s2 = 'birthday'

%time print(edist(s1, s2))

5
CPU times: user 413 µs, sys: 110 µs, total: 523 µs
Wall time: 450 µs


# Problem 3

LeetCode 187: Repeated DNA Sequences (https://leetcode.com/problems/repeated-dna-sequences/)

All DNA is composed of a series of nucleotides abbreviated as A, C, G, and T, for example: "ACGAATTCCG". When studying DNA, it is sometimes useful to identify repeated sequences within the DNA.

Write a function to find all the 10-letter-long sequences (substrings) that occur more than once in a DNA molecule.

In [3]:
def findRepeatedDnaSequences(s: str) -> list:

    dictionary = {}

    output = []
    for i in range(len(s)-10+1):
        current_sequence = s[i:i+10]

        if current_sequence in dictionary:
            dictionary[current_sequence] += 1
        else:
            dictionary[current_sequence] = 1

    for key in dictionary:
        if dictionary[key] > 1:
            output.append(key)
    return output

In [4]:
s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
findRepeatedDnaSequences(s)

['AAAAACCCCC', 'CCCCCAAAAA']

In [5]:
s = "AAAAAAAAAAAA"
findRepeatedDnaSequences(s)

['AAAAAAAAAA']

# Problem 4: 

LeetCode 44: Wildcard Matching (https://leetcode.com/problems/wildcard-matching/)

This is one of the string matching problems.

dynamic programming:

create a matrix __M__ where the letters of the string labels the rows and the letters of the pattern labels the columns, and each element corresponds to the boolean value of whether a particular prefix of pattern matches a particular prefix of the string

```
      '' p[0] p[1] ... p[-1]
''    
s[0]
s[1]
.
.
.
s[-1]

M[i][j] = M[i-1][j-1]                     if p[j] == s[i] or p[j] == ?
          M[i][j-1] (*->'') or M[i-1][j]  if p[j] == *
          False                           if p[j] != s[i]

```




In [2]:
def isMatch(s: str, p: str) -> bool:

    if len(p) == 0:
        if len(s) == 0:
            return True
        return False

    # preprocess p to collapse *
    cp = p[0]
    for i in range(1, len(p)):
        if p[i] == '*' and cp[-1] == '*':
            continue
        cp += p[i]


    ncol = len(cp) + 1
    nrow = len(s) + 1
    mat = [[False] * ncol for i in range(nrow)]

    # fill first row and column
    mat[0][0] = True
    if cp[0] == '*':
        mat[0][1] = True

    for i in range(1, nrow):
        for j in range(1, ncol):

            if cp[j-1] == s[i-1] or cp[j-1] == '?':
                mat[i][j] = mat[i-1][j-1]

            elif cp[j-1] == '*':
                mat[i][j] = mat[i][j-1] or mat[i-1][j]


    return mat[-1][-1]

In [3]:
s = "ho"
p = "**ho"

isMatch(s, p)

True

In [4]:
s = "aa"
p = "a"

isMatch(s, p)

False

In [5]:
s = ""
p = ""

isMatch(s, p)

True

# Problem 5 :

LeetCode 686: Repeated String Match (https://leetcode.com/problems/repeated-string-match/)

Given two strings A and B, find the minimum number of times A has to be repeated such that B is a substring of it. If no such solution, return -1.

In [1]:
def repeatedStringMatch(A: str, B: str) -> int:

    num_repeats = 1

    A_repeats = A

    while len(A_repeats) <= (len(B)+len(A)):    

        if B in A_repeats:                
            return num_repeats

        A_repeats += A
        num_repeats += 1

    if B in A_repeats:                
        return num_repeats

    return -1

In [2]:
A = "abc"
B = "cabcabca"

repeatedStringMatch(A, B)

4

# Problem 6:
    
LeetCode 1143: Longest Common Subsequence (https://leetcode.com/problems/longest-common-subsequence/)

# Problem 7:
    
LeetCode 76:  Minimum Window Substring (https://leetcode.com/problems/minimum-window-substring/)